In [1]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_batches = train_datagen.flow_from_directory(
    '/Users/yillysaurus/PycharmProjects/challenge-mole/assets/Training',
    seed=123,
    target_size=(224,224),
    batch_size=32,
    classes=['Benign', 'Malignant'],
    class_mode='binary')




test_batches = test_datagen.flow_from_directory(
    '/Users/yillysaurus/PycharmProjects/challenge-mole/assets/Testing',
    seed=123,
    target_size=(224,224),
    batch_size=32,
    classes=['Benign', 'Malignant'],
    class_mode='binary')

Found 2321 images belonging to 2 classes.
Found 577 images belonging to 2 classes.


In [33]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D())

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model_trained=model.fit_generator(
        train_batches,
        steps_per_epoch=2000 // 32,
        epochs=5,
        validation_data=test_batches,
        validation_steps=800 // 32)

/Users/yillysaurus/PycharmProjects/kaggle-project/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
62/62 [==============================] - 55s 861ms/step - loss: 0.6109 - accuracy: 0.7009 - val_loss: 0.4949 - val_accuracy: 0.7834
Epoch 2/5
62/62 [==============================] - 63s 1s/step - loss: 0.5413 - accuracy: 0.7786

In [36]:
model.save('second_model.h5')

In [34]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [54]:
from cv2 import cv2
from keras.models import load_model
model = load_model('model/second_model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('/Users/yillysaurus/PycharmProjects/challenge-mole/assets/Testing/Malignant/D93.BMP')

img=cv2.resize(img,(224,224))
img = img.reshape(-1,224, 224,3)

result = model.predict_classes(img)

if result[0] == 1:
    print ('You should see a doctor')
else:
    print('Nothing to worry about')


/Users/yillysaurus/PycharmProjects/kaggle-project/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Nothing to worry about


In [45]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
from keras.models import load_model
import numpy as np
import os
from os import listdir





from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
from keras.models import load_model
import numpy as np


def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(200, 200))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 200, 200, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

# load an image and predict the class
def predict(filename):
    # load the image
    img = load_image(filename)
    # load model
    model = load_model('second_model.h5')
    # predict the class
    result = model.predict(img)
    result = result[0]

    return result

predict('/Users/yillysaurus/PycharmProjects/challenge-mole/static/Malignant.BMP')

array([0.], dtype=float32)